In [13]:
###code sumhow does not find all seq_id the rest has to be found by hand####

from Bio import Entrez

#enter your ENTREZ e-mail
Entrez.email = "example.example@example.com"

# Open the input and output files
with open("seq_id.fasta") as f_in, open("seq_id_taxonomy.fasta", "w") as f_out:
    for line in f_in:
        # Skip lines that don't start with ">"
        if not line.startswith(">"):
            continue
        
        # Remove the ">" symbol and newline character from the ID
        sequence_id = line.strip()[1:]
        
        # Define the search term
        search_term = f"{sequence_id}[Accession]"
        
        # Search for the sequence using Entrez
        handle = Entrez.esearch(db="protein", term=search_term)
        record = Entrez.read(handle)
        
        # Get the list of matching IDs
        id_list = record["IdList"]
        
        if len(id_list) == 0:
            print(f"No record found for sequence ID: {sequence_id}")
            continue
        
        # Use the first ID to fetch the record
        record_handle = Entrez.efetch(db="protein", id=id_list[0], retmode="xml")
        record = Entrez.read(record_handle)
        
        # Extract the organism and taxonomy information
        organism = record[0]["GBSeq_organism"]
        taxonomy = record[0]["GBSeq_taxonomy"]
        
        # Print the organism and taxonomy information
        #print(f"Sequence ID: {sequence_id}")
        #print(f"Organism: {organism}")
        #print(f"Taxonomy: {taxonomy}")
        
        # Write the sequence ID and taxonomy information to the output file
        f_out.write(f">{sequence_id} {organism} {taxonomy}\n")


No record found for sequence ID: XP_005584944.1
No record found for sequence ID: XP_024409245.1
No record found for sequence ID: NXG29166.1
No record found for sequence ID: XP_009901177.1


In [14]:
#sort for vertebrate
with open("seq_id_taxonomy.fasta", "r") as input_file:
    with open("seq_id_vertebrata.fasta", "w") as output_file:
        for line in input_file:
            if "Vertebrata" in line:
                output_file.write(line)


In [15]:
#sort for mammalia
with open("seq_id_taxonomy.fasta", "r") as input_file:
    with open("seq_id_mammalia.fasta", "w") as output_file:
        for line in input_file:
            if "Mammalia" in line:
                output_file.write(line)

In [ ]:
###code to extract all vertebrata Iupred sequences####

# open the input files
with open("seq_id_vertebrata.fasta", "r") as fasta_file, open("output_mod.txt", "r") as output_file:
    # read the seq_id from the fasta file and store them in a set
    seq_ids = set()
    for line in fasta_file:
        if line.startswith(">"):
            seq_id = line.split()[0][1:]
            seq_ids.add(seq_id)
    
    # filter the output file by seq_id and write the matching lines to a new file
    with open("Iupred2_vertebrata.fasta", "w") as output:
        for line in output_file:
            seq_id = line.split(";")[0]
            if seq_id in seq_ids:
                output.write(line)

In [ ]:
###code to extract all mammalian Iupred sequences####

# open the input files
with open("seq_id_mammalia.fasta", "r") as fasta_file, open("output_mod.txt", "r") as output_file:
    # read the seq_id from the fasta file and store them in a set
    seq_ids = set()
    for line in fasta_file:
        if line.startswith(">"):
            seq_id = line.split()[0][1:]
            seq_ids.add(seq_id)
    
    # filter the output file by seq_id and write the matching lines to a new file
    with open("Iupred2_mammalia.fasta", "w") as output:
        for line in output_file:
            seq_id = line.split(";")[0]
            if seq_id in seq_ids:
                output.write(line)

In [ ]:
####code to build the mean of all steps in the new ouput file for vertebrate####


import numpy as np

with open("Iupred2_vertebrata.fasta", "r") as f:####here the modified output file from above is used####
    binary_sequences = []
    for line in f:
        line = line.strip()
        # Skip empty or invalid lines
        if not line or ";" not in line:
            continue
        # Extract binary sequence
        binary_sequence = line.split(";")[1]
        binary_sequences.append(binary_sequence)

    # Calculate the mean at each position over all binary sequences
    means = []
    for i in range(len(binary_sequences[0])):
        values = []
        for binary_sequence in binary_sequences:
            if binary_sequence[i] != "-":
                values.append(int(binary_sequence[i]))
        means.append(np.mean(values))

    # Write means to file
    with open("Iupred2_vertebrata_sum.fasta", "w") as outfile:
        for i, mean in enumerate(means):
            outfile.write(f"{i+1};{mean}\n")

In [ ]:
####code to build the mean of all steps in the new ouput file mammalia####


import numpy as np

with open("Iupred2_mammalia.fasta", "r") as f:####here the modified output file from above is used####
    binary_sequences = []
    for line in f:
        line = line.strip()
        # Skip empty or invalid lines
        if not line or ";" not in line:
            continue
        # Extract binary sequence
        binary_sequence = line.split(";")[1]
        binary_sequences.append(binary_sequence)

    # Calculate the mean at each position over all binary sequences
    means = []
    for i in range(len(binary_sequences[0])):
        values = []
        for binary_sequence in binary_sequences:
            if binary_sequence[i] != "-":
                values.append(int(binary_sequence[i]))
        means.append(np.mean(values))

    # Write means to file
    with open("Iupred2_mammalia_sum.fasta", "w") as outfile:
        for i, mean in enumerate(means):
            outfile.write(f"{i+1};{mean}\n")

In [ ]:
###filter for all vertebrata in original Jaleview data (evolutionary censervation####)

# open the input files
with open("seq_id_vertebrata.fasta", "r") as fasta_file, open("Jalview_edited_length_wo_gap.fasta", "r") as jalview_file:
    # read the seq_id from the fasta file and store them in a set
    seq_ids = set()
    for line in fasta_file:
        if line.startswith(">"):
            seq_id = line.split()[0][1:]
            seq_ids.add(seq_id)
    
    # filter the Jalview file by seq_id and write the matching lines to a new file
    with open("evolutionary_conservation_vertebrata.fasta", "w") as output:
        for line in jalview_file:
            if line.startswith(">"):
                seq_id = line.split()[0][1:]
                if seq_id in seq_ids:
                    output.write(line)
                    for line in jalview_file:
                        if line.startswith(">"):
                            break
                        output.write(line)


In [ ]:
###filter for all mammalia in original Jaleview data (evolutionary censervation####)

# open the input files
with open("seq_id_mammalia.fasta", "r") as fasta_file, open("Jalview_edited_length_wo_gap.fasta", "r") as jalview_file:
    # read the seq_id from the fasta file and store them in a set
    seq_ids = set()
    for line in fasta_file:
        if line.startswith(">"):
            seq_id = line.split()[0][1:]
            seq_ids.add(seq_id)
    
    # filter the Jalview file by seq_id and write the matching lines to a new file
    with open("evolutionary_conservation_mammalia.fasta", "w") as output:
        for line in jalview_file:
            if line.startswith(">"):
                seq_id = line.split()[0][1:]
                if seq_id in seq_ids:
                    output.write(line)
                    for line in jalview_file:
                        if line.startswith(">"):
                            break
                        output.write(line)